# Creating a new op in tensorflow          
#####      [Hands on time - 40min]

### Why its needed?
1) When you cannot express your operation as combination of existing operations
<br>2) When the operation is not available. Example - Sparse Convolution Matrix Multiplication

### Steps
1) Register the new op in c++ file. Basically write an interface. <br>
2) Write the operation's logic.<br>
3) Build the operation to generate its .so file.<br>
4) Set .so path's in LD_LIBRARY_PATH.<br>
5) Test if it works in python.<br>
6) You can write a python wrapper on top of it if you want.

### Lets Begin!

### 1) Register the new op in c++ file. Basically write an interface. 

In [ ]:
#include "tensorflow/core/framework/op.h"
#include "tensorflow/core/framework/shape_inference.h"
#include "tensorflow/core/framework/op_kernel.h"

using namespace tensorflow;

REGISTER_OP("ZeroOut")
    .Input("to_zero: int32")
    .Output("zeroed: int32")
    .SetShapeFn([](::tensorflow::shape_inference::InferenceContext* c) {
      c->set_output(0, c->input(0));
      return Status::OK();
    });

### op.h 
OpRegistryInterface which is responsible for registering the operation.<br>
Before that there is watcher which watches a particular operation is already registered<br>
if yes, then returns non-ok status else registers its returning pointer which is then used further.<br>
https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/framework/op.h

### shape_inference.h
It uses InferenceContext by shape inference function which is used for inferencing/setting the shape of output tensors.
It uses set_output() fucntion which specifies the shape of our output function. In above example shape of input and outputs are same.
https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/framework/shape_inference.h

#### Operation name must be in camel case. Why? Later on!

### 2) Write the operation's logic.

### opKernal.h
opKernal provides base function compute() which needs to be overridden with your implementation. It can be both synchronous and Asynchronous. Its implemented in thread-safe way so that many process can use it concurrently.It takes a input tensor which can be accessed using OpKernelContext*
https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/framework/op_kernel.h

In [ ]:
class ZeroOutOp : public OpKernel {
 public:
  explicit ZeroOutOp(OpKernelConstruction* context) : OpKernel(context) {}

  void Compute(OpKernelContext* context) override {
    // Grab the input tensor
    const Tensor& input_tensor = context->input(0);
    auto input = input_tensor.flat<int32>();

    // Create an output tensor
    Tensor* output_tensor = NULL;
    OP_REQUIRES_OK(context, context->allocate_output(0, input_tensor.shape(), &output_tensor)); // to allocate a new output buffer
    auto output = output_tensor->flat<int32>();

    // Set all but the first element of the output tensor to 0.
    const int N = input.size();
    for (int i = 1; i < N; i++) {
      output(i) = 0;
    }

    // Preserve the first input value if possible.
    if (N > 0) output(0) = input(0);
  }
};

REGISTER_KERNEL_BUILDER(Name("ZeroOut").Device(DEVICE_CPU), ZeroOutOp)

#### REGISTER_KERNEL_BUILDER
Final stage of registration process. This is used for setting the execute mode of the operation. Ex CPU or GPU.<br>
This is used when you have same kernal but one for CPU and one for GPU.

### 3) Build the operation to generate its .so file.

#### 1 - Using gcc

Find the path of your tensorflow include directory using:<br>
<i>import tensorflow as tf; print(tf.sysconfig.get_include())</i><br>
TF_INC= "/usr/local/lib/python2.7/dist-packages/tensorflow/include"<br>
g++ -std=c++11 -shared zero_out.cc -o zero_out.so -fPIC -I $TF_INC -O2

#### 2 - Using Bazel

save BUILD file with following configuration :<br>
load("//tensorflow:tensorflow.bzl", "tf_custom_op_library")<br>
tf_custom_op_library(
    name = "zero_out.so",
    srcs = ["zero_out.cc"],
)

bazel build --config opt //tensorflow/core/user_ops:zero_out.so

### 4) Set .so path's in LD_LIBRARY_PATH.

This is very important or else tensorflow won't be able to load this .so file. <br>
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/lib/python2.7/dist-packages/tensorflow/include/zero_out.so

### 5) Test if it works in python.

In [ ]:
import tensorflow as tf
zero_out_module = tf.load_op_library('zero_out.so')
with tf.Session(''):
  zero_out_module.zero_out([[1, 2], [3, 4]]).eval()

# Prints
array([[1, 0], [0, 0]], dtype=int32)

#### tf.load_op_library

Function to load the dynamic library and register the op with the TensorFlow framework. load_op_library returns a Python module that contains the Python wrappers for the op and the kernel.

To answer this #Operation-name-must-be-in-camel-case.-Why?-Later-on!<br>
Wrapper automatically converts the Title case to snake case complying with PEP8. No additional effort to write a wrapper for.
